<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-tools/blob/1-update-control-vocabulary-column-data-type-automate-display/notebooks/AccessDiagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prerequisites
!pip install git+https://github.com/fair-research/bdbag
!pip install deriva
!pip install 'git+https://github.com/informatics-isi-edu/eye-ai-tools.git'


  Cloning https://github.com/fair-research/bdbag to /tmp/pip-req-build-5tex8q9l
  Running command git clone --filter=blob:none --quiet https://github.com/fair-research/bdbag /tmp/pip-req-build-5tex8q9l
  Resolved https://github.com/fair-research/bdbag to commit 3daccacc7324b5604453b9fbb971e005a7b28cee
  Preparing metadata (setup.py) ... done
  Created wheel for bdbag: filename=bdbag-1.7.1-py2.py3-none-any.whl size=72638 sha256=fa09d0354d4b9a24ae45eb734e38e45304e8b884d2e6766d2f26f52ab082da2c
  Stored in directory: /tmp/pip-ephem-wheel-cache-mfti4lsa/wheels/e6/37/a1/ed335483cf88440ed32b8f918cc20fa31b2ee2c81dd9504f7a
Successfully built bdbag
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.4/210.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
# login to DERIVA via Globus Auth
host = 'www.eye-ai.org' #@param {type:"string"}
!deriva-globus-auth-utils login --no-browser --host {host}

2023-11-16 18:21:21,608 - INFO - No default configuration file found, attempting to create one at: /root/.deriva/config.json
2023-11-16 18:21:22,424 - INFO - NoSavedTokens: No tokens were loaded
Please paste the following URL in a browser:
https://auth.globus.org/v2/oauth2/authorize?client_id=8ef15ba9-2b4a-469c-a163-7fd910c9d111&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=urn%3Aglobus%3Aauth%3Ascope%3Agroups.api.globus.org%3Aview_my_groups_and_memberships+openid+https%3A%2F%2Fauth.globus.org%2Fscopes%2Fwww.eye-ai.org%2Fderiva_all&state=_default&response_type=code&code_challenge=Tyi0dU_hpDSvWBKM5uv0miaAyhPQbruktoVdz47XKd0&code_challenge_method=S256&access_type=online&prefill_named_grant=Login+from+deriva-client+on+1426b266eef2+%5BLinux-5.15.120%2B-x86_64-with-glibc2.35%5D+to+hosts+%5Bwww.eye-ai.org%5D+with+requested+scopes+%5Burn%3Aglobus%3Aauth%3Ascope%3Agroups.api.globus.org%3Aview_my_groups_and_memberships%2C+openid%2C+https%3A%2F%2Fauth.globus.org%2Fscope

Make a connection to the Eye AI catalog and get a reference to the main schema: eye-ai

In [3]:
import pandas as pd
import ml_utils
from deriva.core import ErmrestCatalog, get_credential
protocol = 'https'
catalog_number = 'eye-ai'
credential = get_credential(host)
catalog = ErmrestCatalog(protocol, host, catalog_number, credential)

# Get the path builder interface for this catalog
pb = catalog.getPathBuilder()
eye_ai = pb.schemas['eye-ai']


In [39]:
def find_latest_observation(df: pd.DataFrame):
  latest_encounters = {}
  for index, row in df.iterrows():
    subject_rid = row['Subject_RID']
    date_of_encounter = row['Date_of_Encounter']
    if subject_rid not in latest_encounters or date_of_encounter > latest_encounters[subject_rid]:
      latest_encounters[subject_rid] = date_of_encounter
  for index, row in df.iterrows():
    if row['Date_of_Encounter'] != latest_encounters[row['Subject_RID']]:
      df.drop(index, inplace=True)
  df = df[['RID', 'RCB', 'Image',	'Diagnosis_Vocab',	'Cup/Disk_Ratio',	'Image_Quality_Vocab']]
  return df

def image_tall(dataset_rid: str, diagnosis_tag_rid: str):
  # Get references to tables to start path.
  subject_dataset = eye_ai.Subject_Dataset
  subject = eye_ai.Subject
  image = eye_ai.Image
  observation = eye_ai.Observation
  path = subject_dataset.path

  results = path.filter(subject_dataset.Dataset == dataset_rid)\
    .link(subject, on=subject_dataset.Subject==subject.RID)\
    .link(observation, on=subject.RID==observation.Subject)\
    .link(image, on=observation.RID==image.Observation)\
    .filter(image.Image_Angle_Vocab == angle_two_rid)\
    .link(diagnosis, on=image.RID==diagnosis.Image)\
    .filter(diagnosis.Diagnosis_Tag==diagnosis_tag_rid)

  results = results.attributes(
            results.Subject.RID.alias("Subject_RID"),
            results.Observation.Date_of_Encounter,
            results.Diagnosis.RID,
            results.Diagnosis.RCB,
            results.Diagnosis.Image,
            results.Diagnosis.Diagnosis_Vocab,
            results.Diagnosis.column_definitions['Cup/Disk_Ratio'],
            results.Diagnosis.Image_Quality_Vocab
            )
  image_frame = pd.DataFrame(results.fetch())

  # # Select only the first observation which included in the grading app.
  image_frame = find_latest_observation(image_frame)

  # Show grader name
  users = pb.schemas['public']
  EC = users.ERMrest_Client
  path = EC.path
  User = pd.DataFrame(path.entities().fetch())[['ID', 'Full_Name']]
  image_frame = pd.merge(image_frame, User, how="left", left_on='RCB', right_on='ID' )


  # Now flatten out Diagnosis_Vocab and Image_quality_Vocab
  diagnosis_vocab = pd.DataFrame(eye_ai.Diagnosis_Image_Vocab.entities().fetch())[['RID', "Name"]]
  diagnosis_vocab.columns=['Diagnosis_Vocab', 'Diagnosis']
  image_quality_vocab = pd.DataFrame(eye_ai.Image_Quality_Vocab.entities().fetch())[['RID', "Name"]]
  image_quality_vocab.columns=['Image_Quality_Vocab', 'Image_Quality']

  image_frame = pd.merge(image_frame, diagnosis_vocab, how="left", on='Diagnosis_Vocab')
  image_frame = pd.merge(image_frame, image_quality_vocab, how="left", on='Image_Quality_Vocab')

  if diagnosis_tag_rid == initial_diagnosis_rid:
    image_frame['Full_Name'] = 'Initial Diagnosis'

  return image_frame[['RID', 'Full_Name',	'Image',	'Diagnosis',	'Cup/Disk_Ratio',	'Image_Quality']]

def Reshape_table(init_diag, grader_diag):
  frames = [grader_diag, init_diag]
  long = pd.concat(frames)
  # change data type for control vocab table
  cols = ['Image_Quality','Full_Name', 'Diagnosis']
  for c in cols:
    long[c] = long[c].astype('category')
  wide = pd.pivot(long, index='Image', columns = 'Full_Name', values = 'Diagnosis') #Reshape from long to wide
  return long, wide

In [ ]:
# @title Dataset Tags

pd.DataFrame(eye_ai.Dataset.path.attributes(eye_ai.Dataset.RID, eye_ai.Dataset.Description))

,RID,Description
0,1-EATE,Initial Rotterdam data set. Contains 101442 su...
1,2-0X9M,Rotterdam Train Subset for Cropping Algorithm ...
2,1-MGYM,Rotterdam dataset for evaluation. 20% of [1-EA...
3,1-NRJP,Rotterdam dataset for ML training. 64% of [1-E...
4,1-SQCE,Rotterdam dataset for ML testing. 16% of [1-EA...
5,2-1S12,**LAC Dataset** The initial data set from LAC....
6,2-1S14,**Invalid Dataset** The subjects in this datas...
7,2-277C,**Test Dataset** A subject dataset intended fo...
8,2-277E,**Development Dataset** All of the subjects in...
9,2-277G,**Training Dataset** A subject dataset that is...


In [ ]:
# @title Diagnosis Tags
pd.DataFrame(eye_ai.Diagnosis_Tag.path.attributes(
    eye_ai.Diagnosis_Tag.RID,
    eye_ai.Diagnosis_Tag.Name,
    eye_ai.Diagnosis_Tag.Description))


,RID,Name,Description
0,2-1S0R,UI Annotation,UI Annotation
1,C1T4,Initial Diagnosis,The diagnosis is extracted from the original d...
2,2-35EW,UI Annotation Testing,UI Annotation Testing
3,2-35G0,GlaucomaSuspect,Glaucoma suspect study
4,2-35RM,AI_glaucomasuspect_test,Diagnosis tag for TEST dataset for AI algorith...


In [5]:
# @title Configure
diagnosis = eye_ai.Diagnosis  # All of the diagnosis in the catalog
dataset_rid = "2-277M" # @param {type:"string"}
diagnosis_rid = "2-35RM" # @param {type:"string"}
initial_diagnosis_rid = "C1T4"
angle_two_rid = '2SK6'

dataset = eye_ai.Dataset
ds = dataset.path.filter(dataset.RID==dataset_rid).attributes(dataset.RID, dataset.Description).fetch()[0]
print(f"Dataset tag (RID:{dataset_rid}):\n\tdescription: {ds['Description']}")

diagnosis_tag = eye_ai.Diagnosis_Tag
diagnosis_info = diagnosis_tag.path.filter(diagnosis_tag.RID==diagnosis_rid).attributes(diagnosis_tag.RID, diagnosis_tag.Name, diagnosis_tag.Description).fetch()[0]
print(f"Diagnosis tag (RID:{diagnosis_rid}):\n\tname {diagnosis_info['Name']} \n\tdescription: {diagnosis_info['Description']}")


Dataset tag (RID:2-277M):
	description: **Graded Test Dataset** A random subset of subjects from the [test](https://www.eye-ai.org/id/eye-ai/2-277C) dataset. Contains 250 subjects from each diagnosis category.
Diagnosis tag (RID:2-35RM):
	name AI_glaucomasuspect_test 
	description: Diagnosis tag for TEST dataset for AI algorithm to identify glaucoma suspects in LAC DHS dataset (1000 images from 250 glaucoma suspects and 250 normals)


In [40]:
# @title Example of Graded Test Dataset

Graded_test_initial_diag = image_tall(dataset_rid, initial_diagnosis_rid)
Graded_test_grader_diag = image_tall(dataset_rid, diagnosis_rid)
long, wide = Reshape_table(Graded_test_initial_diag, Graded_test_grader_diag)

long.to_csv("GradedTest_diag_long.csv")
wide.to_csv("GradedTest_diag_wide.csv")